In [249]:
%pip install pyodbc

^C


In [254]:
%pip install plotly==5.6.0

Note: you may need to restart the kernel to use updated packages.


In [330]:
import pyodbc
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [27]:
conn = pyodbc.connect('#see pyodbc documentation')

In [28]:
#think before running (download time > 6 min)
cursor = conn.cursor()
query = "SELECT * FROM table"
df = pd.read_sql(query, conn)

In [29]:
#df.to_csv('year_to_date.csv')
#df = pd.read_csv('year_to_date.csv')

In [359]:
df_opp = pd.read_excel('OPP_unique.xlsx')

In [360]:
df_opp['ArtNr'] = df_opp['ArtNr'].astype('int64')

In [361]:
df2 = df[(df['DokDatum'] > '2021-01-01') & (df['DokDatum'] < '2022-03-31')]
df2['DokDatum'] = pd.to_datetime(df2['DokDatum'], format = '%Y-%m-%d')
df2['YYYY_MM'] = pd.to_datetime(df2['DokDatum']).dt.strftime('%Y-%m')

<ipython-input-361-daad71331ab7>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-361-daad71331ab7>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [362]:
df3 = df2[df2['DokStufe'] == '4']

In [363]:
df4 = df3[df3['Artikelnummer'].str.isnumeric()]
df6 = df3[df3['Artikelnummer'].str.isnumeric() == False]

In [364]:
df4['Artikelnummer'] = df4['Artikelnummer'].astype('int64')

<ipython-input-364-c9d6e7f406a7>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [365]:
df5 = df4.merge(df_opp, how = 'left', left_on = 'Artikelnummer', right_on = 'ArtNr')

In [366]:
df5_opp = df4.merge(df_opp, how = 'inner', left_on = 'Artikelnummer', right_on = 'ArtNr')

In [367]:
df5['DokDatum'] = pd.to_datetime(df5['DokDatum'], format = '%Y-%m-%d')

In [368]:
df7 = df5.append(df6)

In [369]:
df7['OPP'] = df7['OPP'].fillna('Non_OPP')

In [370]:
df7_non_OPP = df7[df7['OPP'] =='Non_OPP']

In [371]:
df8 = pd.pivot_table(df7,index=['DokADSachbearbeiterName', 'OPP'], values = ['UmsatzMIs', 'DBCHFMIS'], aggfunc = np.sum)

In [372]:
df8['DB%'] = df8['DBCHFMIS'] / df8['UmsatzMIs'] 

In [394]:
df8.to_excel('aussendienst_opp_analysis.xlsx')

In [374]:
df9 = pd.pivot_table(df5_opp,index=['Artikelnummer', 'Art.Bezeichnung'], values = ['UmsatzMIs', 'DBCHFMIS'], aggfunc = np.sum)

In [375]:
df9['DB%'] = df9['DBCHFMIS'] / df9['UmsatzMIs']

In [377]:
fig = px.scatter(df_9_non_negative, x = 'UmsatzMIs', y = "DB%",log_x = True)
fig.show()

In [378]:
df10 = pd.pivot_table(df5_opp,index=['YYYY_MM'], values = ['UmsatzMIs', 'DBCHFMIS'], aggfunc = np.sum)
df10['DB%'] = df10['DBCHFMIS'] / df10['UmsatzMIs']

In [379]:
df11 = pd.pivot_table(df7,index=['YYYY_MM'], values = ['UmsatzMIs', 'DBCHFMIS'], aggfunc = np.sum)
df11['DB%'] = df11['DBCHFMIS'] / df11['UmsatzMIs']

In [380]:
df12 = pd.pivot_table(df7_non_OPP,index=['YYYY_MM'], values = ['UmsatzMIs', 'DBCHFMIS'], aggfunc = np.sum)
df12['DB%'] = df12['DBCHFMIS'] / df12['UmsatzMIs']
df12['% Total Umsatz'] = df12['UmsatzMIs'] / df11['UmsatzMIs']

In [381]:
df10['% Total Umsatz'] = df10['UmsatzMIs'] / df11['UmsatzMIs']

In [382]:
df10 = df10.rename(columns = {'DB%': 'DB% (OPP)', '% Total Umsatz': '% Total Umsatz (OPP)'})
df12 = df12.rename(columns = {'DB%': 'DB% (non_OPP)', '% Total Umsatz': '% Total Umsatz (non_OPP)'})
df11 = df11.rename(columns = {'DB%': 'DB% (total)'})

In [398]:
df13 = pd.concat([df10, df12, df11], axis = 1)
df13 = df13[['DB% (OPP)', 'DB% (non_OPP)', 'DB% (total)', '% Total Umsatz (OPP)', '% Total Umsatz (non_OPP)']]

In [400]:
df13.to_excel(db_opp_analysis.xlsx)

,DB% (OPP),DB% (non_OPP),DB% (total),% Total Umsatz (OPP),% Total Umsatz (non_OPP)
YYYY_MM,,,,,
2021-01,0.359864,0.314445,0.324349,0.218069,0.781931
2021-02,0.348196,0.297627,0.305001,0.145826,0.854174
2021-03,0.344637,0.283478,0.290600,0.116448,0.883552
2021-04,0.308530,0.296750,0.298498,0.148346,0.851654
2021-05,0.352011,0.301226,0.307556,0.124642,0.875358
2021-06,0.350119,0.386145,0.382509,0.100934,0.899066
2021-07,0.372651,0.301275,0.309967,0.121780,0.878220
2021-08,0.361819,0.309553,0.315888,0.121215,0.878785
2021-09,0.362672,0.298550,0.306052,0.117002,0.882998


In [392]:
df14 = df13.reset_index()
fig2 = make_subplots(specs = [[{'secondary_y': False}]])
fig2.add_trace(
    go.Scatter(x = df14['YYYY_MM'], y = df14['DB% (OPP)'], name = 'DB% (OPP)')
)
fig2.add_trace(
    go.Scatter(x = df14['YYYY_MM'], y = df14['DB% (non_OPP)'], name = 'DB% (non_OPP)')
)
fig2.add_trace(
    go.Scatter(x = df14['YYYY_MM'], y = df14['DB% (total)'], name = 'DB% (total)')
)
fig2.show()